In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import requests
import dotenv
import os
import json
from bs4 import BeautifulSoup


In [2]:
ct = contrans()
ct.mypassword
congresskey= ct.congresskey

In [3]:
useragent = ct.get_useragent()
useragent


'python-requests/2.32.3'

In [18]:
members = ct.get_bioguideIDs()
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,L000604,4.0,"Lopez, Greg",Republican,Colorado,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star..."
1,M001225,15.0,"Mullin, Kevin",Democratic,California,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star..."
2,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star..."
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]"
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...,"[{'chamber': 'House of Representatives', 'star..."
...,...,...,...,...,...,...,...,...,...,...
34,B001267,NaN,"Bennet, Michael F.",Democratic,Colorado,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]"
35,B001243,NaN,"Blackburn, Marsha",Republican,Tennessee,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY..."
36,B001236,NaN,"Boozman, John",Republican,Arkansas,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY..."
37,B001230,NaN,"Baldwin, Tammy",Democratic,Wisconsin,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [4]:
headers=ct.make_headers()
headers

{'User-Agent': 'python-requests/2.32.3', 'From': 'mathaimadelyn@gmail.com'}

In [22]:
cands = pd.read_csv('data/cands22.txt', quotechar="|", header=None)
cands.columns = ['Cycle', 'FECCandID', 'CID','FirstLastP','Party','DistIDRunFor','DistIDCurr','CurrCand','CycleCand','CRPICO','RecipCode','NoPacs']
print(cands)

      Cycle  FECCandID        CID                 FirstLastP Party  \
0      2022  H0AK00105  N00039029            Thomas Lamb (3)     3   
1      2022  H0AL01055  N00044245             Jerry Carl (R)     R   
2      2022  H0AL01063  N00044288            Wes Lambert (R)     R   
3      2022  H0AL01097  N00044750         James Averhart (D)     D   
4      2022  H0AL02087  N00030768            Martha Roby (R)     R   
...     ...        ...        ...                        ...   ...   
8923   2022  S8WI00224  N00041929            Leah Vukmir (R)     R   
8924   2022  S8WV00119  N00041293  Paula Jean Swearengin (D)     D   
8925   2022  S8WV00127  N00035531           Evan Jenkins (R)     R   
8926   2022  S8WV00135  N00041474              Bo Copley (R)     R   
8927   2022  S8WV00143  N00012642       Patrick Morrisey (R)     R   

     DistIDRunFor DistIDCurr CurrCand CycleCand CRPICO RecipCode NoPacs  
0            AK01                                             3N         
1          

In [44]:
replace_map = {'Republican': 'R',
                'Democratic': 'D', 
                'Independent': 'I'}

members['partyletter']= members['partyName'].replace(replace_map)
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item,lastname,firstname,name2,partyletter
0,L000604,4.0,"Lopez, Greg",Republican,Colorado,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star...",Lopez,Greg,Greg Lopez (Republican),R
1,M001225,15.0,"Mullin, Kevin",Democratic,California,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star...",Mullin,Kevin,Kevin Mullin (Democratic),D
2,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star...",McIver,LaMonica,LaMonica McIver (Democratic),D
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]",Helmy,George S.,George S. Helmy (Democratic),D
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...,"[{'chamber': 'House of Representatives', 'star...",Ruiz,Raul,Raul Ruiz (Democratic),D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,B001267,NaN,"Bennet, Michael F.",Democratic,Colorado,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]",Bennet,Michael F.,Michael F. Bennet (Democratic),D
35,B001243,NaN,"Blackburn, Marsha",Republican,Tennessee,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY...",Blackburn,Marsha,Marsha Blackburn (Republican),R
36,B001236,NaN,"Boozman, John",Republican,Arkansas,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY...",Boozman,John,John Boozman (Republican),R
37,B001230,NaN,"Baldwin, Tammy",Democratic,Wisconsin,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY...",Baldwin,Tammy,Tammy Baldwin (Democratic),D


In [35]:
members['lastname'] = [x.split(',')[0] for x in members['name']]
members['firstname'] = [x.split(',')[1] for x in members['name']]
members['name2']= [x +' '+y + ' (' +z + ')' for x,y,z in 
                   zip(members['firstname'], members['lastname'], members['partyletter'])]
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item,lastname,firstname,name2,partyletter
0,L000604,4.0,"Lopez, Greg",Republican,Colorado,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star...",Lopez,Greg,Greg Lopez (Republican),Republican
1,M001225,15.0,"Mullin, Kevin",Democratic,California,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star...",Mullin,Kevin,Kevin Mullin (Democratic),Democratic
2,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star...",McIver,LaMonica,LaMonica McIver (Democratic),Democratic
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]",Helmy,George S.,George S. Helmy (Democratic),Democratic
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...,"[{'chamber': 'House of Representatives', 'star...",Ruiz,Raul,Raul Ruiz (Democratic),Democratic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,B001267,NaN,"Bennet, Michael F.",Democratic,Colorado,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]",Bennet,Michael F.,Michael F. Bennet (Democratic),Democratic
35,B001243,NaN,"Blackburn, Marsha",Republican,Tennessee,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY...",Blackburn,Marsha,Marsha Blackburn (Republican),Republican
36,B001236,NaN,"Boozman, John",Republican,Arkansas,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY...",Boozman,John,John Boozman (Republican),Republican
37,B001230,NaN,"Baldwin, Tammy",Democratic,Wisconsin,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY...",Baldwin,Tammy,Tammy Baldwin (Democratic),Democratic


In [45]:
crosswalk = pd.merge(members, cands, left_on='name2',
                     right_on='FirstLastP', 
                     how='outer', 
                     indicator= 'matched',
                     validate='one_to_one')

crosswalk

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

In [25]:
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,L000604,4.0,"Lopez, Greg",Republican,Colorado,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star..."
1,M001225,15.0,"Mullin, Kevin",Democratic,California,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star..."
2,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star..."
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]"
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...,"[{'chamber': 'House of Representatives', 'star..."
...,...,...,...,...,...,...,...,...,...,...
34,B001267,NaN,"Bennet, Michael F.",Democratic,Colorado,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]"
35,B001243,NaN,"Blackburn, Marsha",Republican,Tennessee,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY..."
36,B001236,NaN,"Boozman, John",Republican,Arkansas,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY..."
37,B001230,NaN,"Baldwin, Tammy",Democratic,Wisconsin,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [ ]:
# What are we matching on?

# What kind of merge is this? One to one?

#

In [5]:
print(ct.get_bioguideIDs)


<bound method contrans.get_bioguideIDs of <contrans.contrans object at 0x10fd0c3b0>>


In [6]:
goodbioguide = ct.get_bioguide(name = 'good', state = 'Virginia', district = 5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."


In [7]:
goodbioguide['bioguideId'].reset_index()

,index,bioguideId
0,0,G000595


In [8]:
goodid = goodbioguide['bioguideId'].reset_index()['bioguideId'][0]

In [9]:
goodid

'G000595'

In [10]:
#Sponsored legislation
goodsbills = ct.get_sponsoredlegislation(goodid)


In [11]:
r = requests.get(goodsbills[0]['url'],
                 params = {'api_key':congresskey}) #goodsbills[0]['url']

texturl = json.loads(r.text)['bill']['textVersions']['url']


r = requests.get(texturl,
                 params= {'api_key':congresskey})
json.loads(r.text)['textVersions'][0]['formats'][0]['url']

toscrape = json.loads(r.text)['textVersions'][0]['formats'][0]['url']

In [12]:
from bs4 import BeautifulSoup

r = requests.get(toscrape)
mysoup = BeautifulSoup(r.text, 'html.parser') #equivalent of json.loads, allows you to search through a string
billtext = mysoup.text

In [13]:
billurl = goodsbills[0]['url']

In [14]:
ct.get_billdata(billurl)

In [15]:
print(mysoup.text)


[Congressional Bills 118th Congress]
[From the U.S. Government Publishing Office]
[H. Res. 1509 Introduced in House (IH)]

<DOC>






118th CONGRESS
  2d Session
H. RES. 1509

Supporting the designation of the week beginning November 11, 2024, as 
  ``National Pregnancy Center Week'' to recognize the vital role that 
community-supported pregnancy centers play in saving lives and serving 
        women and men faced with difficult pregnancy decisions.


_______________________________________________________________________


                    IN THE HOUSE OF REPRESENTATIVES

                           September 25, 2024

 Mr. Good of Virginia (for himself, Mr. Clyde, Mr. Mooney, Mr. Gosar, 
 Mr. Posey, Mr. Burgess, Mr. Bacon, Mr. Fleischmann, Mr. Grothman, Mr. 
Rose, Mr. Fulcher, Mr. Biggs, Mr. Higgins of Louisiana, Mr. Ogles, Mr. 
 Kelly of Pennsylvania, Mr. Lamborn, Mrs. Harshbarger, Mr. Norman, Mr. 
  Wilson of South Carolina, Mrs. Miller of Illinois, Mr. LaMalfa, Mr. 
    Murph

In [16]:
votes = ct.get_votes()
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [17]:
house = house(['rollnumber', 'icpsr','cast_code'])

NameError: name 'house' is not defined

In [ ]:
house_mat = pd.merge(house, house on = 'rollnumber')
house_mat

In [ ]:
house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree': 'mean'})
house_mat = house_mat.reset_index()
house_mat

In [ ]:
ideo = ct.get_ideology()
members = ideo[['bioname', 'icpsr','partycode']]
members

In [ ]:
agreement = pd.merge(house_mat, members, 
                     left_on = 'icpsr_x', 
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member1',
                              'party_code':'party1'}, axis=1)
agreement = agreement.drop(['icspr_x'],axis=1)
agreement = pd.merge(agreement, members,
                     left_on = 'icspr_y',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member2',
                              'party_code':'party2'}, axis=1)
agreement = agreement[['member1','party1','member2','party2','agree']]
agreement

In [ ]:
agreement[agreement['member1'].str.contains('GOOD,')].sort_values('agree', ascending = False).head(30)